In [57]:
import polars as pl
from typing import List

customer = pl.read_excel(
    "./data/excel data/Maintain Customer.xlsx",
    sheet_name="Maintain Customer",
)
supplier = pl.read_excel(
    "./data/excel data/Maintain Supplier.xlsx",
    sheet_name="Maintain Supplier",
)
sal_pur = pl.read_excel(
    "./data/excel data/bst jan2025 sales&pur ffb.xlsx",
    sheet_name="jan2025",
)

sal_pur = sal_pur.filter(
    (sal_pur["Date Out"].is_not_null()) & (sal_pur["Supplier"].is_not_null())
)

sal_pur = sal_pur.join(supplier, left_on="Supplier", right_on="Code", how="inner")
# only need columns [Date Out => DocDate, Supplier => Code, Net Wt(Ton) => Qty, Price(ton) => U/Price]
sal_pur = sal_pur.select([
    "Date Out",
    "Supplier",
    "Net Wt(Ton)",
    "Price(ton)",
]).rename({
    "Date Out": "DocDate",
    "Supplier": "Code",
    "Net Wt(Ton)": "Qty",
    "Price(ton)": "U/Price",
})
# sal_pur = sal_pur.with_row_index(offset=1)



Could not determine dtype for column 7, falling back to string


In [58]:
start_index = 2
sal_pur_grouped = (
    sal_pur
    .group_by(
        ["Code"],
        maintain_order=True
    )
    .all()
    .with_row_index(offset=start_index)
    .with_columns(
        Seq = pl.col("DocDate").list.len().map_elements(lambda x: list(range(1, x + 1)), return_dtype=List[int]),
        # DocNo = f"PI-{pl.col('index'):0>5}"
        DocNo = pl.col("index").map_elements(lambda x: f"PI-{x:0>5}")
    )
    .drop(["index"])
    .explode("DocDate", "Seq", "Qty", "U/Price")
)
sal_pur_grouped


sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


Code,DocDate,Qty,U/Price,Seq,DocNo
str,date,f64,f64,i64,str
"""1015""",2025-01-16,0.96,900.0,1,"""PI-00002"""
"""1015""",2025-01-20,0.82,880.0,2,"""PI-00002"""
"""1026""",2025-01-23,0.36,880.0,1,"""PI-00003"""
"""2019""",2025-01-04,0.81,920.0,1,"""PI-00004"""
"""2019""",2025-01-26,1.08,880.0,2,"""PI-00004"""
…,…,…,…,…,…
"""2114""",2025-01-11,2.01,900.0,2,"""PI-00044"""
"""2114""",2025-01-12,0.05,900.0,3,"""PI-00044"""
"""2114""",2025-01-13,0.02,900.0,4,"""PI-00044"""


In [68]:
# record of supplier that is not in the supplier table

sal_pur_ = pl.read_excel(
    "./data/excel data/bst jan2025 sales&pur ffb.xlsx",
    sheet_name="jan2025",
)
supplier_ = pl.read_excel(
    "./data/excel data/Maintain Supplier.xlsx",
    sheet_name="Maintain Supplier",
)
supplier_ = supplier_.with_columns(
    Supplier_Code = pl.col("Code")
)
sal_pur_.join(supplier_, left_on="Supplier", right_on="Code", how="left").filter(pl.col("Supplier_Code").is_null())


Could not determine dtype for column 7, falling back to string


Date Out,Serial No,Lorry No,Supplier,Net Wt(Ton),Price(ton),Gross Amt,TPT CHRG,Tpt Amt,30%,Worker Chrg,Worker Amt,Others,NET,Company Name,Control A/C,Company Category,Address 1,Address 2,Address 3,Address 4,Post Code,TIN,ID Type,ID No,Supplier_Code
date,str,str,str,f64,f64,f64,i64,f64,f64,i64,f64,i64,f64,str,str,str,str,str,str,str,str,str,str,str,str
2025-01-22,"""P0028959""","""3357""","""1001""",0.12,880.0,105.6,null,0.0,null,null,0.0,null,105.6,null,null,null,null,null,null,null,null,null,null,null,null
2025-01-22,"""P0028960""","""SA3357T""","""1001""",0.05,880.0,44.0,null,0.0,null,null,0.0,null,44.0,null,null,null,null,null,null,null,null,null,null,null,null
2025-01-09,"""P0028861""","""WMH5591""","""1004""",0.16,920.0,147.2,null,0.0,null,null,0.0,null,147.2,null,null,null,null,null,null,null,null,null,null,null,null
2025-01-10,"""P0028866""","""WH5591""","""1004""",0.17,900.0,153.0,null,0.0,null,null,0.0,null,153.0,null,null,null,null,null,null,null,null,null,null,null,null
2025-01-21,"""P0028943""","""4867""","""1005""",0.25,880.0,220.0,null,0.0,null,null,0.0,null,220.0,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
null,null,null,null,2411.695299,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,"""sales:""","""1. tian siang""",28.56,977.84,27927.1104,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,"""2. d&b tdg""",245.18,998.03,244696.9954,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [ ]:
import fastexcel as fex

sal_pur = fex.read_excel("./data/excel data/bst jan2025 sales&pur ffb.xlsx")
print(sal_pur.sheet_names)



['Supplier', 'Purchase (2)', 'jan2025', 'PURCHASE']
